<a href="https://colab.research.google.com/github/louis8ok1/DCGAN/blob/main/keras_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function, division
from keras.datasets import mnist #手寫辨識
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np

In [ ]:
class DCGAN():
  def __init__(self):
    #Input Shape
    self.img_shape = (28,28,1)
    self.latent_dim = 100
    optimizer = Adam(0.0002,0.5)

    #Build and compile the discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',optimzer = optimizer,metrics={'output_discriminator'：'accuracy'}) 

    #Build the generator
    self.generator = self.build_generator()

    #The generator takes noise as input and generates fake imgs
    z = Input(shape(self.latent_dim,))#z的dim為100
    img = self.generator(z)

    # For the combined model we will only train the generator
    # 大問題!!!!!!!!!!(因為前面已經compile)
    self.discriminator.trainable = False

    # The discriminator takes generated images as input and determines validity
    valid = self.discriminator(img)

    #The combined model
    #Trains the generator to fool the discriminator
    self.combined = Model(z,valid)
    self.combined.compile(loss='binary_crossentropy',optimzer = optimizer,metrics={'output_generator'：'accuracy'})
    
  def build_generator(self):
    model = Sequential()
    model.add(Dense(128*7*7,activation='relu',input_dim=self.latent_dim))
    model.add(Reshape((7,7,128)))
    model.add(UpSampling2D())
    model.add(Cov2D(64,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    #test是不是比較好
    model.add(UpSampling2D())
    model.add(Conv2D(64,kernel_size=3,padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(Conv2D(1,kernel_size=3,padding='same'))
    model.add(Activation('tanh'))

    model.summary()

    noise = Input(shape=(self.latent_dim,))
    img = model(noise)

    return Model(noise,img)

  def build_discriminator(self):

    model = Sequential()
    model.add(Conv2D(32,kernel_size=3,strides=2,input_dim=self.img_shape,padding='same'))
    #alpha = Negative slope coefficient
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,kernel_size=3,strides=2,padding='same'))
    #在四周都填0
    model.add(ZeroPadding2D(padding((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,kernel_size=3,strides=2,padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    #FC layer
    model.add(Conv2D(256,kernel_size=3,strides=1,padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))

    model.summary()

    img = Input(shape=self.img_shape)
    validity = model(img)

    return Model(img,validity)
    #batch_size????????
  def train(self,epochs,batch_size = 128,save_interval=50):

    #Load data
    (X_train,_),(_,_)=mnist.load_data()

    #normalization
    X_train








